In [ ]:
%matplotlib inline

import json, os
import numpy as np
import subprocess as spc

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import ipywidgets as widgets
from ipywidgets import interact

from water_automata import ulti_1_step,ulti_2_step

In [ ]:
from plotly import graph_objs as plgo

---

In [ ]:
### 1
SIZE = 100
TIME = 1000
DELTA_T = 0.001
HH = 20/SIZE

GG = 9.8
SPEED = 1.0

In [ ]:
# Initial state:
water1 = np.ones((TIME+1, SIZE, SIZE))*2

xs1, ys1 = np.meshgrid(*([np.linspace(0, SIZE*HH, 2*SIZE+1)[1:-1:2]])*2)
gaussian1 = np.exp( -((xs1-SIZE*HH/4)**2 + (ys1-SIZE*HH/4)**2) )

water1[0,:,:] += gaussian1*4

mo_x = np.zeros((TIME+1, SIZE, SIZE))
mo_y = np.zeros((TIME+1, SIZE, SIZE))

In [ ]:
# Iterate:
for t in range(TIME):
    water1[t+1],mo_x[t+1],mo_y[t+1] = ulti_2_step(water1[t],mo_x[t],mo_y[t],
        d_t=DELTA_T,d_x=HH,gg=GG,vv=SPEED)

---

In [ ]:
### 2
SIZE = 50
TIME = 1000
DELTA_T = 0.001
HH = 20/SIZE

GG = 9.8
SPEED = 1.0

In [ ]:
# Initial state 2:
water2 = np.ones((TIME+1, SIZE, SIZE))*2

xs2, ys2 = np.meshgrid(*([np.linspace(0, SIZE*HH, 2*SIZE+1)[1:-1:2]])*2)
gaussian2 = np.exp( -((xs2-SIZE*HH/4)**2 + (ys2-SIZE*HH/4)**2) )

water2[0,:,:] += gaussian2*4

mo_x = np.zeros((TIME+1, SIZE, SIZE))
mo_y = np.zeros((TIME+1, SIZE, SIZE))

In [ ]:
# Iterate 2:
for t in range(TIME):
    water2[t+1],mo_x[t+1],mo_y[t+1] = ulti_2_step(water2[t],mo_x[t],mo_y[t],
        d_t=DELTA_T,d_x=HH,gg=GG,vv=SPEED)

---

In [ ]:
fig = plgo.FigureWidget()

surf1 = plgo.Surface()
surf1.x = xs1
surf1.y = ys1
surf1.z = water1[0]
surf1.colorscale = [
    (0, '#4000AA'),
    (0.2, '#0000FF'),
    (0.4, '#0040FF'),
    (0.6, '#0080FF'),
    (0.8, '#00BFFF'),
    (1, '#00FFFF'),
]
surf1.scene = 'scene1'
surf1.colorbar.showticklabels = False

fig.add_trace(surf1)

surf2 = plgo.Surface()
surf2.x = xs2
surf2.y = ys2
surf2.z = water2[0]
surf2.colorscale = [
    (0, '#4000AA'),
    (0.2, '#0000FF'),
    (0.4, '#0040FF'),
    (0.6, '#0080FF'),
    (0.8, '#00BFFF'),
    (1, '#00FFFF'),
]
surf2.scene = 'scene2'
surf2.colorbar.showticklabels = False

fig.add_trace(surf2)

fig.layout.scene1.zaxis.range = (0, 6)
fig.layout.scene1.aspectmode = 'cube'
fig.layout.scene1.domain.x = (0, 0.48)

fig.layout.update(scene2=plgo.layout.Scene())
fig.layout.scene2.zaxis.range = (0, 6)
fig.layout.scene2.aspectmode = 'cube'
fig.layout.scene2.domain.x = (0.52, 1)

fig.layout.height = 450
fig.layout.width = 900

slider = widgets.IntSlider(**{
    'min': 0,
    'value': 0,
    'max': TIME,
    'step': 10
})

def link1(scene, camera, *args):
    with fig.batch_update():
        fig.layout.scene2.camera.update({
            'center': camera.center,
            'eye': camera.eye,
            'up': camera.up,
        })
        
def link2(scene, camera, *args):
    with fig.batch_update():
        fig.layout.scene1.camera.update({
            'center': camera.center,
            'eye': camera.eye,
            'up': camera.up,
        })
    
fig.layout.scene1.on_change(link1, 'camera')
fig.layout.scene2.on_change(link2, 'camera')

def move_water(change):
    with fig.batch_animate():
        fig.data[0].z = water1[change['new']]
        fig.data[1].z = water2[change['new']]

slider.observe(move_water, names='value')

widgets.VBox([slider, fig])